# *Depot-D.NLR*

### Extracting Text From PDF

In [3]:
import os      
import fitz    
from langchain.text_splitter import RecursiveCharacterTextSplitter


def extract_text_from_pdfs(project_dataset):
    """Extracts text from all PDFs in a given folder."""
    all_text_chunks = []  

    for pdf_file in os.listdir(project_dataset): 
        if pdf_file.endswith(".pdf"):  
            pdf_path = os.path.join(project_dataset, pdf_file)  
            doc = fitz.open(pdf_path)  
            
            for page_num, page in enumerate(doc): 
                text = page.get_text("text")  
                if text.strip():  
                    all_text_chunks.append({  
                        "content": text,  
                        "metadata": {"source": pdf_file, "page": page_num + 1}  
                    })
    
    return all_text_chunks  

### Chunking Them Into Documents

In [4]:
from langchain_core.documents import Document


def process_pdf(pdf_path):
    """Extracts text from a PDF and splits it into smaller chunks with metadata."""
    pdf_texts = extract_text_from_pdfs(pdf_path)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    
    # all_chunks = []
    documents= []
    for text_entry in pdf_texts:
        
        chunks = text_splitter.split_text(text_entry["content"])
        
        pagenumber = text_entry["metadata"]["page"]
        metadata = text_entry["metadata"]

        for idx, chunk in enumerate(chunks):
            # print(chunk[:10])
            # print(metadata)
            # print(f"{pagenumber}-{idx}")

            doc = Document(
                page_content=chunk,
                metadata=metadata,
                id=f"{pagenumber}-{idx}"
                
            )

            # print(doc)
            documents.append(doc)
            # all_chunks.append({"content": chunk, "metadata": text_entry["metadata"]})

    return documents
    
    
documents = process_pdf("./")

### Embedding Function / Embedding Setup for the Chunks / Documents

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\RAG-llm\D-RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting up The ChromaDB Vector Store

In [6]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="Depot-D.NLR_db",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

### Add Documents to The Vector Store

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['e2f47ecc-16de-4ad0-ab75-ac4ff9ad7b0c',
 '65175820-89dc-45f5-9f55-e97280f4283e',
 'b5e95ae0-c57d-487c-a8e1-26b4d305e2f1',
 'f64b1ce3-d49d-4c8e-b408-5d3b83863f2f',
 'f1e12643-1d8c-4d09-b96c-f7ac79e0b411',
 'ca58520e-ecfe-4a46-aa62-809daa3e1d42',
 'db5613ea-988b-4bab-b7bf-8386de887d6c',
 'fc65fa2b-90ee-42b1-b661-25e0bb0379be',
 '628dcd66-b640-43a1-a67d-2edd1d26cc3f',
 '3f34c739-89df-4cdd-9faf-90f277848eb7',
 'dab0dda1-1a9a-423b-8e08-4ecf46b7da0a',
 'eda2edb6-b558-4736-9df6-0d2ad86bea57',
 'bf5e00b3-925e-4c5f-97a8-d40c849fe48a',
 '71e80e4d-93f1-4eaa-b17e-402588ce099a',
 'b60990b3-37e7-47aa-9fc1-6ee4e7029408',
 '4bbc292b-b638-4792-92e7-df16559112d1',
 'af6200b4-ea09-4de7-87a2-649aec3a8369',
 'c39755e3-a850-456e-965d-83d8b4c8836c',
 '5ff5c8ff-816b-47fa-90fd-8c761f9549f1',
 'a9c820ab-5926-4784-a7f6-70af1b2f1dc9',
 '29936308-7547-4d06-80b1-5ea22d8dbb59',
 'ee36a3ca-c83e-4331-a657-d23452ae8a5d',
 'ae8c1359-c4c9-4406-bd8c-8cb922c4f249',
 'e1bbfbc9-d254-490e-be9c-5f9faafa1e2d',
 '04a6fc17-9a55-

### Retriever Setup / Retriever Function

In [8]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10}
)

### LLM Model

In [9]:
from langchain_community.llms import Ollama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Optional: for streaming output
callbacks = [StreamingStdOutCallbackHandler()]

llm = Ollama(
    model="llama3.1",  # this should match the model name shown in `ollama list`
    temperature=0.01,
    top_p=0.95,
    repeat_penalty=1.03,
    callbacks=callbacks,
)

# Test it
#response = llm.invoke("What is Retrieval Augmented Generation?")
#print(response)


C:\Users\ronit\AppData\Local\Temp\ipykernel_23332\3453628631.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


### Response Cleaner Function

In [10]:
def clean_response(text):
    text = text.strip()
    if text.lower().startswith("train accident is"):
        text = text[text.find('.') + 1:].strip()
    
    keywords = ["include", "types of", "are as follows"]
    for word in keywords:
        if word in text.lower():
            text = text.replace("•", "-")
            break

    return text


### Prompt Template

In [11]:
from langchain_core.prompts import PromptTemplate

SYSTEM_TEMPLATE = """
You are an assistant specialized in Indian Railways documentation. 
Answer the following question using only the provided context. 
Be concise, skip generic definitions, and prioritize information relevant to Indian Railways.

If the answer involves categories, list them clearly in bullet points or numbered format.
Always include source metadata like PDF name and page number if available.

Question: {question}

Context:
{context}
    
    """

question_answering_prompt = PromptTemplate(template=SYSTEM_TEMPLATE, input_variables=["question", "context"])

### THE RAG CHAIN

In [12]:

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def format_docs(docs):
    context = "\n\n".join(doc.page_content for doc in docs)
    print(f"----n{context}\n----")
    return context

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | question_answering_prompt
    | llm
    | StrOutputParser()
)

### Qyerying

In [13]:
query = "What is the procedure for train accident investigation?"
response = rag_chain.invoke(query)

----nthe section where provided with signals etc.  
vii. prima facie cause of accident;  
g)  ensure that the Train Signal Register, Log Book, Private Number Book, Line Admission 
Book, speed Recorder Chart and other relevant records are seized,  
h)  obtain the statements of staff involved in the accident, as far as possible;  
i)   prepare a rough sketch showing the position of derailed vehicles, position of track and 
OHE etc. make a quick survey of the extent of damage ;

they have to be performed. On occurrence of an accident to his train, the Guard of the train 
shall immediately:  
1. Note the time of accident. 
2. Make a quick survey of the accident site for casualties, injuries, if any, and for deciding 
the assistance required.  
3. Send the first information of accident to the control office and then to the nearest 
Station Master furnishing the following information, through mobile phone or portable 
telephone, or walkie-talkie or gate phone, etc.,

Accident inquiries done 

### Cleaning the Response

In [14]:
# Clean the LLM response
cleaned = clean_response(response)
# Add metadata from the top 2-3 retrieved documents
docs = retriever.get_relevant_documents(query)
sources = []
for doc in docs:
    meta = doc.metadata
    sources.append(f"[Source: {meta.get('source', 'unknown')} | Page: {meta.get('page', '?')}]")
# Remove duplicates
sources = list(set(sources))
# Final formatted output
final_output = cleaned + "\n\n" + "\n".join(sources)
print(final_output)

C:\Users\ronit\AppData\Local\Temp\ipykernel_23332\3692205166.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


The procedure for train accident investigation in Indian Railways involves the following steps:

**Initial Steps**

1. Note the time of accident (Source: PDF, page 14)
2. Make a quick survey of the accident site for casualties, injuries, and assistance required (Source: PDF, page 14)
3. Send the first information of accident to the control office and nearest Station Master (Source: PDF, page 14)

**Investigation**

* Seize relevant records such as Train Signal Register, Log Book, Private Number Book, Line Admission Book, speed Recorder Chart, and other relevant records (Source: PDF, page vii)
* Obtain statements of staff involved in the accident (Source: PDF, page vii)
* Prepare a rough sketch showing the position of derailed vehicles, track, and OHE (Source: PDF, page vii)

**Additional Steps**

* In case of serious accidents, the Commissioner of Railway Safety conducts the inquiry (Source: PDF, page 14)
* In case of other accidents, departmental officials conduct the inquiry (Source: